In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Load API Key from .env
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")


# Initialize Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

# Chatbot
chat_memory = ConversationBufferMemory()
chat_chain = ConversationChain(llm=llm, memory=chat_memory)
def chat_fn(message, history): return chat_chain.run(message)

# Summarizer
summary_template = PromptTemplate.from_template("Summarize this news article:\n\n{article}")
summary_chain = LLMChain(llm=llm, prompt=summary_template)
def summarize_fn(article): return summary_chain.run(article=article)

# Translator
translator_template = PromptTemplate.from_template("Translate the following text to {language}:\n\n{text}")
translator_chain = LLMChain(llm=llm, prompt=translator_template)
def translate_fn(text, language): return translator_chain.run(text=text, language=language)

# Resume Analyzer
resume_template = PromptTemplate.from_template("Analyze this resume and suggest improvements:\n\n{resume}")
resume_chain = LLMChain(llm=llm, prompt=resume_template)
def resume_fn(resume): return resume_chain.run(resume=resume)

# Medical Diagnosis
med_template = PromptTemplate.from_template("Given these symptoms: {symptoms}, provide a possible medical diagnosis and advice.")
med_chain = LLMChain(llm=llm, prompt=med_template)
def med_fn(symptoms): return med_chain.run(symptoms=symptoms)

# Gradio Tab 1: AI Chatbot
chat_ui = gr.ChatInterface(fn=chat_fn, title="💬 AI Chatbot")

# Gradio Tab 2: News Summarizer
summarizer_ui = gr.Interface(
    fn=summarize_fn,
    inputs=gr.Textbox(placeholder="Paste a news article...", lines=10),
    outputs="textbox",
    title="📰 News Summarizer"
)

# Gradio Tab 3: Language Translator
translator_ui = gr.Interface(
    fn=translate_fn,
    inputs=[
        gr.Textbox(label="Text to Translate", lines=5),
        gr.Dropdown(["French", "Spanish", "Hindi", "German"], label="Target Language")
    ],
    outputs="textbox",
    title="🌍 Language Translator"
)

# Gradio Tab 4: Resume Analyzer
resume_ui = gr.Interface(
    fn=resume_fn,
    inputs=gr.Textbox(label="Paste Resume", lines=15),
    outputs="textbox",
    title="📄 Resume Analyzer"
)

# Gradio Tab 5: Medical Diagnosis Assistant
med_ui = gr.Interface(
    fn=med_fn,
    inputs=gr.Textbox(label="Describe your symptoms", lines=8),
    outputs="textbox",
    title="🏥 Medical Diagnosis Assistant"
)

# Combine all into tabs
gr.TabbedInterface(
    interface_list=[chat_ui, summarizer_ui, translator_ui, resume_ui, med_ui],
    tab_names=["Chatbot", "Summarizer", "Translator", "Resume Analyzer", "Medical Assistant"]
).launch(share=True)
